<a href="https://colab.research.google.com/github/youngjooo/DCC-project/blob/main/model/gernerateface_with_styleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 설정

In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
# git clone : github에 공개된 styleGAN2 clone(복제).
!git clone https://github.com/NVlabs/stylegan2

Cloning into 'stylegan2'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 131 (delta 0), reused 1 (delta 0), pack-reused 127
Receiving objects: 100% (131/131), 593.76 KiB | 17.99 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [ ]:
!python /content/stylegan2/run_generator.py generate-images \
--network=gdrive:networks/stylegan2-ffhq-config-f.pkl \
 --seeds=4-6 --truncation-psi=0.4 #원하는 갯수 #1에 가까울수록 더 현실적

Local submit - run_dir: results/00000-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...
2021-08-16 02:09:18.223843: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Failed!
Traceback (most recent call last):
  File "/content/stylegan2/run_generator.py", line 168, in <module>
    main()
  File "/content/stylegan2/run_generator.py", line 163, in main
    dnnlib.submit_run(sc, func_name_map[subcmd], **kwargs)
  File "/content/stylegan2/dnnlib/submission/submit.py", line 343, in submit_run
    return farm.submit(submit_config, host_run_dir)
  File "/content/stylegan2/dnnlib/submission/internal/local.py", line 22, in submit
    return run_wrapper(submit_config)
  File "/content/stylegan2/dnnlib/submission/submit.py", line 280, in run_wrapper


In [ ]:
!ffmpeg -r 1 -i /content/results/00000-generate-images/seed%04d.png -vcodec mpeg4 -y people.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
!cp people.mp4 /content

cp: 'people.mp4' and '/content/people.mp4' are the same file


# generate face

In [ ]:
import sys
sys.path.insert(0, "/content/stylegan2")

In [ ]:
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
import pretrained_networks

In [ ]:
def seeder(seeds, vector_size):
 result = []
 for seed in seeds:
   rnd = np.random.RandomState(seed)
   result.append( rnd.randn(1, vector_size))
 return result

In [ ]:
def generate_images(gs, seeds, truncation_psi):
   noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]
 
   Gs_kwargs = dnnlib.EasyDict()
   Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
   Gs_kwargs.randomize_noise = False
   if truncation_psi is not None:
       Gs_kwargs.truncation_psi = truncation_psi
 
   for seed_idx, seed in enumerate(seeds):
       print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
       rnd = np.random.RandomState()
       tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
       images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
       path = f"/content/out/image{seed_idx}.png"
       PIL.Image.fromarray(images[0], 'RGB').save(path)

In [ ]:
network_pkl = "gdrive:networks/stylegan2-ffhq-config-f.pkl"
print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)

Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.


In [ ]:
vector_size = Gs.input_shape[1:][0]
seeds = seeder(range(6, 5), vector_size)

In [ ]:
!mkdir /content/out

In [ ]:
seeds = seeder([6, 5], vector_size)

In [ ]:
STEPS = 300
diff = seeds[1] - seeds[0]
step = diff / STEPS
current = seeds[0].copy()
 
seeds2 = []
for i in range(STEPS):
 seeds2.append(current)
 current = current + step

In [ ]:
for i in range(STEPS):
 seeds2.append(current)
 current = current + step

In [ ]:
generate_images(Gs, seeds2, truncation_psi=0.4)

Generating image for seed 0/600 ...
Generating image for seed 1/600 ...
Generating image for seed 2/600 ...
Generating image for seed 3/600 ...
Generating image for seed 4/600 ...
Generating image for seed 5/600 ...
Generating image for seed 6/600 ...
Generating image for seed 7/600 ...
Generating image for seed 8/600 ...
Generating image for seed 9/600 ...
Generating image for seed 10/600 ...
Generating image for seed 11/600 ...
Generating image for seed 12/600 ...
Generating image for seed 13/600 ...
Generating image for seed 14/600 ...
Generating image for seed 15/600 ...
Generating image for seed 16/600 ...
Generating image for seed 17/600 ...
Generating image for seed 18/600 ...
Generating image for seed 19/600 ...
Generating image for seed 20/600 ...
Generating image for seed 21/600 ...
Generating image for seed 22/600 ...
Generating image for seed 23/600 ...
Generating image for seed 24/600 ...
Generating image for seed 25/600 ...
Generating image for seed 26/600 ...
Generating 

In [ ]:
!ffmpeg -r 50 -i /content/out/image%d.png -vcodec mpeg4 -y faces.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
!cp faces.mp4 /content

cp: 'faces.mp4' and '/content/faces.mp4' are the same file
